In [13]:
# !pip install yfinance
# !pip install pytickersymbols
# !pip install finta

In [14]:
import os
import requests 
from io import StringIO
import yfinance
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pytickersymbols import PyTickerSymbols
from datetime import datetime, timedelta
from tqdm.notebook import tqdm
from time import sleep
import random 
import plotly.graph_objects as go

from finta import TA

risk = 1.5

In [15]:
stock_data = PyTickerSymbols()

nasdaq_100=pd.DataFrame(stock_data.get_stocks_by_index('NASDAQ 100'))
nasdaq_100 = nasdaq_100[nasdaq_100.symbol != 'CTRP'].reset_index().symbol.to_list()

sp_500=pd.DataFrame(stock_data.get_stocks_by_index('S&P 500'))
sp_500 = sp_500[sp_500.symbol != 'SSE']
sp_500 = sp_500[sp_500.symbol != 'RTN']
sp_500 = sp_500[sp_500.symbol != 'CXO']
sp_500 = sp_500[sp_500.symbol != 'JEC']
sp_500 = sp_500[sp_500.symbol != 'SPY']
sp_500 = sp_500[sp_500.symbol != 'TIF'].reset_index().symbol.to_list()

stocks = nasdaq_100 + sp_500 #+ sp400

stocks = set(stocks) #Convierte la lista a Set, que lo único que hace es quitar elementos repetidos
stocks = list(stocks) #Reconvertir el set a list (para poder iterar sobre ella)

In [16]:
#stocks = random.sample(stocks, 50)

In [17]:
len(stocks)

517

In [18]:
df_complete = pd.DataFrame()
lista = []
for t in tqdm(range(len(stocks))):
  #OPCION PARA DATOS CON YFINANCE ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
  data_day = yfinance.download(stocks[t], period='30d')[['Open', 'Close', 'Volume']]
  #data_day['Ticker'] = stocks[t]
  #data_day['Gap'] = np.nan
  for i in range(1,len(data_day)): 
    gap = (data_day['Open'][i] - data_day['Close'][i-1]) / data_day['Close'][i-1]
    #data_day['Gap'][i] = gap
    if (gap >= .04 or gap <= -.04) and data_day.Volume[i] > 750000:
      lista.append([data_day.index[i], stocks[t], gap])

  #df_complete = pd.concat([df_complete,data_day])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [19]:
len(lista)

40

In [20]:
lista

[[Timestamp('2021-06-29 00:00:00'), 'TXT', 0.041666713971942444],
 [Timestamp('2021-06-11 00:00:00'), 'VRTX', -0.10485773187400269],
 [Timestamp('2021-07-09 00:00:00'), 'NOV', 0.0535965626834377],
 [Timestamp('2021-06-15 00:00:00'), 'ARE', -0.042835701640172],
 [Timestamp('2021-06-24 00:00:00'), 'BIIB', -0.05899436085366833],
 [Timestamp('2021-06-25 00:00:00'), 'KMX', 0.0523319097716618],
 [Timestamp('2021-05-28 00:00:00'), 'HPQ', -0.051090326093647616],
 [Timestamp('2021-07-08 00:00:00'), 'ENPH', -0.047736940258270716],
 [Timestamp('2021-06-04 00:00:00'), 'OGN', 0.07333732774116238],
 [Timestamp('2021-07-01 00:00:00'), 'OXY', 0.05052759812049002],
 [Timestamp('2021-07-09 00:00:00'), 'DFS', 0.052056219753565135],
 [Timestamp('2021-06-01 00:00:00'), 'MRO', 0.04872008004249403],
 [Timestamp('2021-07-01 00:00:00'), 'MRO', 0.041850198209514966],
 [Timestamp('2021-06-15 00:00:00'), 'FCX', -0.04052681085797354],
 [Timestamp('2021-07-08 00:00:00'), 'FCX', -0.05992814757832561],
 [Timestamp('2

In [21]:
target_counter = 0
stop_counter = 0
green_counter = 0
red_counter = 0

for element in lista:
  example = yfinance.download(element[1], start=(element[0] - timedelta(days=1)).strftime('%Y-%m-%d'), 
                           end=(element[0] + timedelta(days=1)).strftime('%Y-%m-%d'), interval='2m')#[['Open', 'High', 'Low', 'Close']]

  example['EMA10'] = TA.EMA(example, 10)
  #example['PriceVol'] = example.Close * example.Volume
  #example['VWAP'] = TA.VWAP(example)
  #example['VWAP'] = (example.PriceVol.rolling(int(len(example)/2)).sum()) / example.Volume.rolling(int(len(example)/2)).sum()
  
  for i in range(len(example)):
    if example.index[i].strftime('%Y-%m-%d') == element[0].strftime('%Y-%m-%d') and len(example) > 200:
      #Para Gap up
      if element[2] > 0 and example.Low[i] <= example.EMA10[i]:
        #Si entra, convertir el high en entry (short) y low en target, y modificarlo dependiendo de como abra el siguiente candle
        entry = example.High[i] + .01 
        target = example.Low[i] - .01
        stop = entry + (entry-target) 
        for j in range(i+1, len(example)):
          #Entra si...
          if example.High[j] > entry:
            print(j)
            for k in range(j+1, len(example)):
              #Se para (el trade lol) si...
              if example.Low[k] <= target:
                target_counter += 1
                print(k)
                print(target)
                print(entry)
                print(stop)
                print('TARGET')
                break
              elif example.High[k] >= stop:
                stop_counter += 1
                print(k)
                print(target)
                print(entry)
                print(stop)
                print('STOP')
                break
            break
          #Si no entra, cambiar el entry y target...
          else:
            entry = example.High[j] + .01
            target = min(stop,(example.Low[j] - .01))  
            stop = entry + (entry-target)  
          #break
        break
      #Para Gap down  
      # elif element[2] < 0 and example.High[i] >= example.EMA10[i]:
      #   #Si entra, convertir el high en target y low en stop (gap-up), y modificarlo dependiendo de como abra el siguiente candle
      #   entry = example.Low[i] - .01
      #   stop = example.High[i] + .01
      #   target = entry - (risk *(stop-entry)) 
      #   for j in range(i+1, len(example)):
      #     #Entra si...
      #     if example.Low[j] < entry:
      #       print(j)
      #       for k in range(j+1, len(example)):
      #         #Se para (el trade lol) si...
      #         if example.Low[k] <= target:
      #           target_counter += 1
      #           print(k)
      #           print(stop)
      #           print(entry)
      #           print(target)
      #           print('TARGET')
      #           break
      #         elif example.High[k] >= stop:
      #           stop_counter += 1
      #           print(k)
      #           print(stop)
      #           print(entry)
      #           print(target)
      #           print('STOP')
      #           break
      #       break
      #     #Si no entra, cambiar el entry y target...
      #     else:
      #       entry = example.Low[j] - .01
      #       stop = max(stop,(example.High[j] + .01))
      #       target = entry - (risk *(stop-entry))      
      #     #break
      #   break
  
  if len(example) > 200:
    fig = go.Figure(data=[go.Candlestick(open=example.Open, high=example.High,low=example.Low, close=example.Close),
                          go.Scatter(y=example.EMA10, line=dict(color='blue', width=1))])
    fig.update_layout(xaxis_rangeslider_visible=False)
    fig.update_layout(title=element[1] + ' on ' + element[0].strftime('%Y-%m-%d'))
    fig.show()

[*********************100%***********************]  1 of 1 completed
210
218
69.33999847412109
69.52000213623047
69.70000579833986
TARGET


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
219
226
14.790000190734863
14.860000381469726
14.93000057220459
STOP


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


...

[*********************100%***********************]  1 of 1 completed
218
219
151.59000610351563
152.01999450683593
152.44998291015622
TARGET


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
214
224
138.48000549316407
139.23999572753905
139.99998596191404
STOP


[*********************100%***********************]  1 of 1 completed
203
204
344.18000244140626
346.1199853515625
348.0599682617187
STOP


[*********************100%***********************]  1 of 1 completed
203
204
21.809999694824217
21.960000762939455
22.110001831054692
STOP


In [22]:
target_counter

6

In [23]:
stop_counter

15

In [24]:
#ZONA DE PRUEBAS
example = yfinance.download('TAL', start='2021-06-16', 
                          end='2021-06-18', interval='2m', prepost = False)#[['Open', 'High', 'Low', 'Close']]

example['EMA10'] = TA.EMA(example, 10)
example['Vol_Act'] = example.Volume
example.Vol_Act.iloc[195] = example.Volume[195] * 0.7
example.Vol_Act.iloc[196:] = example.Volume[196:] * 0.95
example['PriceVol'] = example.Close * example.Vol_Act
#example['VWAP'] = TA.VWAP(example)
example['VWAP'] = (example.PriceVol.rolling(195).sum()) / example.Volume.rolling(195).sum()

fig = go.Figure(data=[go.Candlestick(open=example.Open, high=example.High,low=example.Low, close=example.Close),
                      go.Scatter(y=example.EMA10, line=dict(color='blue', width=1)),
                      go.Scatter(y=example.VWAP, line=dict(color='yellow', width=1))])
fig.update_layout(xaxis_rangeslider_visible=False)
#fig.update_layout(title=element[1] + ' on ' + element[0].strftime('%Y-%m-%d'))
fig.show()

[*********************100%***********************]  1 of 1 completed


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

